In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [3]:
###load the scaler, model, and encoder
model = load_model('model.h5')

##load encoder
with open('label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('ohe.pkl', 'rb') as file:
    ohe = pickle.load(file)

In [4]:
input_data = { 
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 10000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
## One hot encode geography
geography_encoded = ohe.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geography_encoded, columns=ohe.get_feature_names_out(['Geography']))
geo_encoded_df


/Users/gaurishmaheshwari/Documents/Churn Project/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_Germany,Geography_Spain
0,0.0,0.0


In [12]:
input_df = pd.DataFrame([input_data])
input_df = input_df.drop(columns=['Geography'])
input_df = pd.concat([input_df, geo_encoded_df], axis=1)

In [13]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,600,Male,40,5,10000,2,1,1,50000,0.0,0.0


In [14]:
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df 

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,600,1,40,5,10000,2,1,1,50000,0.0,0.0


In [15]:
##scaling the data
input_df_scaled = scaler.transform(input_df)

In [16]:
input_df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.00134472, -1.05836066,
         0.80843615,  0.64920267,  0.97481699, -0.87683221, -0.57946723,
        -0.57638802]])

In [17]:
prediction = model.predict(input_df_scaled)
prediction

1/1 [==============================] - 0s 101ms/step


array([[0.01573127]], dtype=float32)

In [18]:
prediction_proba = prediction[0][0]

if prediction_proba > 0.5:
    result = "The customer is likely to churn."
else:
    result = "The customer is not likely to churn."

print(result)

The customer is not likely to churn.
